To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**NEW** Unsloth now supports training the new **gpt-oss** model from OpenAI! You can start finetune gpt-oss for free with our **[Colab notebook](https://x.com/UnslothAI/status/1953896997867729075)**!

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/llama-2-13b-bnb-4bit",
    "unsloth/codellama-34b-bnb-4bit",
    "unsloth/tinyllama-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
    "unsloth/gemma-2b-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/tinyllama-bnb-4bit", # "unsloth/tinyllama" for 16bit loading
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.4: Fast Llama patching. Transformers: 4.55.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: unsloth/tinyllama-bnb-4bit can only handle sequence lengths of at most 2048.
But with kaiokendev's RoPE scaling of 2.0, it can be magically be extended to 4096!
<string>:37: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.


model.safetensors:   0%|          | 0.00/762M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/948 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

**[NOTE]** TinyLlama's internal maximum sequence length is 2048. We use RoPE Scaling to extend it to 4096 with Unsloth!

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

**[NOTE]** We set `gradient_checkpointing=False` ONLY for TinyLlama since Unsloth saves tonnes of memory usage. This does NOT work for `llama-2-7b` or `mistral-7b` since the memory usage will still exceed Tesla T4's 15GB. GC recomputes the forward pass during the backward pass, saving loads of memory.

`**[IF YOU GET OUT OF MEMORY]**` set `gradient_checkpointing` to `True`.

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Currently only supports dropout = 0
    bias = "none",    # Currently only supports bias = "none"
    use_gradient_checkpointing = False, # @@@ IF YOU GET OUT OF MEMORY - set to True @@@
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.8.4 patched 22 layers with 22 QKV layers, 22 O layers and 22 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_v0.3_(7B)-Conversational.ipynb)

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

In [10]:
# Z-GO Dataset Preparation dengan Hugging Face Go Datasets - FIXED VERSION
# Modifikasi untuk mengatasi error batch processing

from datasets import load_dataset, concatenate_datasets
import random

# Setup tokenizer (pastikan sudah load model dulu)
# model, tokenizer = FastLanguageModel.from_pretrained(...)

EOS_TOKEN = tokenizer.eos_token

print("Starting Z-GO Dataset Preparation...")
print("="*60)

# Dataset loading dengan error handling yang lebih baik
datasets_info = []


# Dataset 2: Golang Coder (skip yang bermasalah untuk sementara)
print("\nLoading golang-coder...")
try:
    dataset_coder = load_dataset("smcleod/golang-coder", split="train")
    print(f"✓ Loaded {len(dataset_coder)} examples from golang-coder dataset")
    print(f"Columns: {dataset_coder.column_names}")
    # datasets_info.append(('coder', dataset_coder))  # Comment dulu karena error
    print("⚠️  Skipping golang-coder due to batch processing issues")
    dataset_coder = None  # Skip for now
except Exception as e:
    print(f"❌ Error loading golang-coder dataset: {e}")
    dataset_coder = None

# Dataset 3: Code-Text Go (Semeru)
print("\nLoading code-text-go...")
try:
    dataset_semeru = load_dataset("semeru/code-text-go", split="train")
    print(f"✓ Loaded {len(dataset_semeru)} examples from code-text-go dataset")
    print(f"Columns: {dataset_semeru.column_names}")
    datasets_info.append(('semeru', dataset_semeru))
except Exception as e:
    print(f"❌ Error loading code-text-go dataset: {e}")
    dataset_semeru = None

# Dataset 4: CodeXGLUE Code-to-Text (Go subset)
print("\nLoading CodeXGLUE...")
try:
    dataset_codexglue = load_dataset(
        "google/code_x_glue_ct_code_to_text",
        "go",
        split="train"
    )
    print(f"✓ Loaded {len(dataset_codexglue)} examples from CodeXGLUE Go dataset")
    print(f"Columns: {dataset_codexglue.column_names}")
    datasets_info.append(('codexglue', dataset_codexglue))
except Exception as e:
    print(f"❌ Error loading CodeXGLUE dataset: {e}")
    dataset_codexglue = None


# Inspect dataset structures
print("\n" + "="*60)
print("DATASET STRUCTURE ANALYSIS")
print("="*60)

def analyze_dataset(dataset, name):
    if dataset is None:
        return

    print(f"\n--- {name} ---")
    print(f"Size: {len(dataset)}")
    print(f"Columns: {dataset.column_names}")

    # Show first example
    if len(dataset) > 0:
        first_example = dataset[0]
        print("First example keys:", list(first_example.keys()))
        for key, value in first_example.items():
            if isinstance(value, str):
                preview = value[:100] + "..." if len(value) > 100 else value
                print(f"  {key}: {preview}")
            elif isinstance(value, list) and len(value) > 0:
                if isinstance(value[0], str):
                    preview = str(value)[:100] + "..." if len(str(value)) > 100 else str(value)
                    print(f"  {key}: {preview}")
                else:
                    print(f"  {key}: List with {len(value)} items of type {type(value[0])}")
            else:
                print(f"  {key}: {type(value)} - {str(value)[:50]}")

# Analyze each loaded dataset
for name, dataset in datasets_info:
    analyze_dataset(dataset, name.title())

# FIXED formatting functions dengan better error handling
def format_best_practices(examples):
    """Format golang-programming-style-best-practices dataset - FIXED"""
    texts = []
    batch_size = len(list(examples.values())[0])  # Get batch size from first column

    try:
        keys = list(examples.keys())
        print(f"Best practices keys: {keys}")

        # Try different field combinations
        if "code" in keys and "explanation" in keys:
            for i in range(batch_size):
                code = examples["code"][i] if i < len(examples["code"]) else ""
                explanation = examples["explanation"][i] if i < len(examples["explanation"]) else ""
                if code or explanation:
                    text = f"// {explanation}\n{code}" + EOS_TOKEN
                    texts.append(text)
                else:
                    texts.append("// Example Go code" + EOS_TOKEN)  # Fallback

        elif "text" in keys:
            for i in range(batch_size):
                text = examples["text"][i] if i < len(examples["text"]) else ""
                if text:
                    texts.append(text + EOS_TOKEN)
                else:
                    texts.append("// Example Go code" + EOS_TOKEN)  # Fallback
        else:
            # Use first available string field
            string_field = None
            for key in keys:
                if isinstance(examples[key][0], str):
                    string_field = key
                    break

            if string_field:
                for i in range(batch_size):
                    text = examples[string_field][i] if i < len(examples[string_field]) else ""
                    if text:
                        texts.append(text + EOS_TOKEN)
                    else:
                        texts.append("// Example Go code" + EOS_TOKEN)  # Fallback
            else:
                # Complete fallback
                texts = ["// Example Go code" + EOS_TOKEN] * batch_size

        # Ensure we always return the correct batch size
        while len(texts) < batch_size:
            texts.append("// Example Go code" + EOS_TOKEN)

        return {"text": texts[:batch_size]}  # Ensure exact batch size

    except Exception as e:
        print(f"Error in format_best_practices: {e}")
        # Return safe fallback
        return {"text": ["// Example Go code" + EOS_TOKEN] * batch_size}

def format_golang_coder_safe(examples):
    """Format golang-coder dataset with SAFE batch processing"""
    batch_size = len(list(examples.values())[0])
    texts = []

    try:
        # Check if this is the problematic messages format
        if "messages" in examples:
            messages_list = examples["messages"]

            for i in range(batch_size):
                try:
                    if i < len(messages_list):
                        messages = messages_list[i]

                        # Handle different message formats
                        if isinstance(messages, list) and len(messages) > 0:
                            # Chat format messages
                            formatted_text = ""
                            for msg in messages:
                                if isinstance(msg, dict):
                                    role = msg.get("role", "")
                                    content = msg.get("content", "")
                                    if role == "system":
                                        formatted_text += f"// System: {content}\n"
                                    elif role == "user":
                                        formatted_text += f"// User: {content}\n"
                                    elif role == "assistant":
                                        formatted_text += f"{content}\n"

                            if formatted_text.strip():
                                texts.append(formatted_text + EOS_TOKEN)
                            else:
                                texts.append("// Go code example" + EOS_TOKEN)
                        else:
                            texts.append("// Go code example" + EOS_TOKEN)
                    else:
                        texts.append("// Go code example" + EOS_TOKEN)

                except Exception as inner_e:
                    print(f"Error processing message {i}: {inner_e}")
                    texts.append("// Go code example" + EOS_TOKEN)
        else:
            # Handle other formats
            texts = ["// Go code example" + EOS_TOKEN] * batch_size

        # Ensure exact batch size
        while len(texts) < batch_size:
            texts.append("// Go code example" + EOS_TOKEN)

        return {"text": texts[:batch_size]}

    except Exception as e:
        print(f"Error in format_golang_coder_safe: {e}")
        return {"text": ["// Go code example" + EOS_TOKEN] * batch_size}

def format_semeru_code_text_safe(examples):
    """Format semeru/code-text-go dataset - SAFE version"""
    batch_size = len(list(examples.values())[0])
    texts = []

    try:
        if "code" in examples and "docstring" in examples:
            for i in range(batch_size):
                code = examples["code"][i] if i < len(examples["code"]) else ""
                docstring = examples["docstring"][i] if i < len(examples["docstring"]) else ""

                if code:
                    if docstring and docstring.strip():
                        text = f"{docstring}\n{code}" + EOS_TOKEN
                    else:
                        text = code + EOS_TOKEN
                    texts.append(text)
                else:
                    texts.append("// Go function example" + EOS_TOKEN)
        else:
            texts = ["// Go function example" + EOS_TOKEN] * batch_size

        while len(texts) < batch_size:
            texts.append("// Go function example" + EOS_TOKEN)

        return {"text": texts[:batch_size]}

    except Exception as e:
        print(f"Error in format_semeru_code_text_safe: {e}")
        return {"text": ["// Go function example" + EOS_TOKEN] * batch_size}

def format_codexglue_safe(examples):
    """Format CodeXGLUE dataset - SAFE version"""
    batch_size = len(list(examples.values())[0])
    texts = []

    try:
        if "code" in examples and "docstring" in examples:
            for i in range(batch_size):
                code = examples["code"][i] if i < len(examples["code"]) else ""
                docstring = examples["docstring"][i] if i < len(examples["docstring"]) else ""

                if code:
                    if docstring and docstring.strip():
                        text = f"{docstring}\n{code}" + EOS_TOKEN
                    else:
                        text = code + EOS_TOKEN
                    texts.append(text)
                else:
                    texts.append("// Go function example" + EOS_TOKEN)
        else:
            texts = ["// Go function example" + EOS_TOKEN] * batch_size

        while len(texts) < batch_size:
            texts.append("// Go function example" + EOS_TOKEN)

        return {"text": texts[:batch_size]}

    except Exception as e:
        print(f"Error in format_codexglue_safe: {e}")
        return {"text": ["// Go function example" + EOS_TOKEN] * batch_size}

# Apply formatting to each dataset with better error handling
formatted_datasets = []

print("\n" + "="*60)
print("FORMATTING DATASETS")
print("="*60)

for name, dataset in datasets_info:
    if dataset is None:
        continue

    print(f"\n🔄 Formatting {name} dataset...")
    try:
        if name == 'practices':
            formatted = dataset.map(format_best_practices, batched=True, batch_size=100)
        elif name == 'coder':
            formatted = dataset.map(format_golang_coder_safe, batched=True, batch_size=100)
        elif name == 'semeru':
            formatted = dataset.map(format_semeru_code_text_safe, batched=True, batch_size=100)
        elif name == 'codexglue':
            formatted = dataset.map(format_codexglue_safe, batched=True, batch_size=100)
        else:
            continue

        formatted_datasets.append(formatted)
        print(f"✓ Successfully formatted {len(formatted)} examples from {name}")

        # Show sample
        if len(formatted) > 0:
            sample = formatted[0]["text"][:200]
            print(f"Sample: {sample}...")

    except Exception as e:
        print(f"❌ Error formatting {name} dataset: {e}")
        continue

# Combine datasets
if formatted_datasets:
    print(f"\n🔗 Combining {len(formatted_datasets)} datasets...")
    final_dataset = concatenate_datasets(formatted_datasets)
    print(f"✅ Final combined dataset size: {len(final_dataset)}")

    # Shuffle
    final_dataset = final_dataset.shuffle(seed=42)
    print("✅ Dataset shuffled")

    # Limit for Colab
    MAX_EXAMPLES = 10000
    if len(final_dataset) > MAX_EXAMPLES:
        final_dataset = final_dataset.select(range(MAX_EXAMPLES))
        print(f"✅ Limited to {MAX_EXAMPLES} examples for efficient training")

    # Show samples
    print(f"\n📝 Sample from final dataset:")
    print("-" * 80)
    for i in range(min(3, len(final_dataset))):
        sample = final_dataset[i]["text"]
        preview = sample[:300] + "..." if len(sample) > 300 else sample
        print(f"Example {i+1}:\n{preview}\n")
        print("-" * 80)

    print(f"\n✅ SUCCESS! Dataset ready with {len(final_dataset)} examples")
    print("Variable name: final_dataset")

else:
    print("❌ No datasets could be processed successfully")
    # Create minimal fallback dataset
    fallback_examples = [
        "// Hello World in Go\npackage main\n\nimport \"fmt\"\n\nfunc main() {\n    fmt.Println(\"Hello, World!\")\n}" + EOS_TOKEN,
        "// Function to add two numbers\nfunc add(a, b int) int {\n    return a + b\n}" + EOS_TOKEN,
        "// Struct definition\ntype Person struct {\n    Name string\n    Age  int\n}" + EOS_TOKEN
    ]

    from datasets import Dataset
    final_dataset = Dataset.from_dict({"text": fallback_examples})
    print(f"Created fallback dataset with {len(final_dataset)} examples")

print("\n" + "="*60)
print("Z-GO DATASET PREPARATION COMPLETE!")
print("="*60)
print("Ready for training with SFTTrainer!")

# Training code
training_code = '''
# Training setup:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = final_dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 200,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "z-go-outputs",
        save_steps = 50,
        save_total_limit = 2,
    ),
)

trainer_stats = trainer.train()
'''

print(training_code)

Starting Z-GO Dataset Preparation...

Loading golang-coder...
✓ Loaded 304692 examples from golang-coder dataset
Columns: ['messages']
⚠️  Skipping golang-coder due to batch processing issues

Loading code-text-go...
✓ Loaded 167288 examples from code-text-go dataset
Columns: ['repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'partition']

Loading CodeXGLUE...
❌ Error loading CodeXGLUE dataset: BuilderConfig 'code-to-text-go' not found. Available: ['go', 'java', 'javascript', 'php', 'python', 'ruby']
✓ Loaded 167288 examples from alternative CodeXGLUE
Columns: ['id', 'repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url']

DATASET STRUCTURE ANALYSIS

--- Semeru ---
Size: 167288
Columns: ['repo', 'path', 'func_name', 'original_string', 'language', 'code', 'code_tokens', 'docstring', 'docstring_tokens', 'sha', 'url', 'partition']
First 

Map:   0%|          | 0/167288 [00:00<?, ? examples/s]

✓ Successfully formatted 167288 examples from semeru
Sample: // getAllDepTypes returns a sorted list of names of all dep type
// commands.
func getAllDepTypes() []string {
	depTypes := make([]string, 0, len(cmds))
	for depType := range cmds {
		depTypes = appen...

🔄 Formatting codexglue dataset...


Map:   0%|          | 0/167288 [00:00<?, ? examples/s]

✓ Successfully formatted 167288 examples from codexglue
Sample: // getStringValue will return a quoted string and the amount
// of bytes read
//
// an error will be returned if the string is not properly formatted
func getStringValue(b []rune) (int, error) {
	if b...

🔗 Combining 2 datasets...
✅ Final combined dataset size: 334576
✅ Dataset shuffled
✅ Limited to 10000 examples for efficient training

📝 Sample from final dataset:
--------------------------------------------------------------------------------
Example 1:
// PgDescriptionByObjoidClassoidObjsubid retrieves a row from 'pg_catalog.pg_description' as a PgDescription.
//
// Generated from index 'pg_description_o_c_o_index'.
func PgDescriptionByObjoidClassoidObjsubid(db XODB, objoid pgtypes.Oid, classoid pgtypes.Oid, objsubid int) (*PgDescription, error) {...

--------------------------------------------------------------------------------
Example 2:
// ReadByte works like bufio.ReadByte
func (c *Conn) ReadByte() (b byte, err e

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 1 full epoch which makes Alpaca run in 80ish minutes! We also support TRL's `DPOTrainer`! See our DPO tutorial on a free Google Colab instance [here](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Zephyr_(7B)-DPO.ipynb).

In [ ]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = final_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = True, # Packs short sequences together to save time!
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 1,
        learning_rate = 2e-5,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.1,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading file tokenizer.model from cache at /root/.cache/huggingface/hub/models--unsloth--tinyllama-bnb-4bit/snapshots/fc56510003ea9d49362400b8a362345150802c31/tokenizer.model
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--unsloth--tinyllama-bnb-4bit/snapshots/fc56510003ea9d49362400b8a362345150802c31/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--unsloth--tinyllama-bnb-4bit/snapshots/fc56510003ea9d49362400b8a362345150802c31/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--unsloth--tinyllama-bnb-4bit/s

Generating train split: 0 examples [00:00, ? examples/s]

Using auto half precision backend


In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
0.879 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

***** Running training *****
  Num examples = 3,000
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 4
  Total optimization steps = 375
  Number of trainable parameters = 25,231,360
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,2.280100
2,2.243700
3,2.245500
4,2.243600
5,2.302000
6,2.287400
7,2.283900
8,2.144600
9,2.237200
10,2.245700




Training completed. Do not forget to share your model on huggingface.co/models =)




In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

5034.6413 seconds used for training.
83.91 minutes used for training.
Peak reserved memory = 13.508 GB.
Peak reserved memory for training = 12.629 GB.
Peak reserved memory % of max memory = 91.592 %.
Peak reserved memory for training % of max memory = 85.632 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nContinue the fibonnaci sequence.\n\n### Input:\n1, 1, 2, 3, 5, 8\n\n### Response:\nThe fibonacci sequence is a sequence of numbers that can be generated by adding the previous two numbers and then subtracting the previous number from the previous number. The first number in the sequence is 1, and the second number is 1. The third number in the sequence is 1 + 1 = 2, the fourth number is 1 + 2 = 3, and so on. The sequence can be continued by adding 1 + 1 = 2, 1 + 2 = 3, and so on.</s>']

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 64)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
